In [10]:
!pip install keras
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 385.2 kB/s eta 0:00:0000:0100:15
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 933.4 kB/s eta 0:00:00:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 573.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 633.8 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 672.9 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 669.1 kB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 173.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 933.9 kB/s eta 0:00:0000:010:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 430.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.

In [14]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MLP tf in PySpark").getOrCreate()

data = spark.read.format("csv").option("header", "true").load("./ml_model/fraudulent_transactions_anonymous_ia.csv")

In [17]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StringIndexer

# Convert target column to numeric using StringIndexer
label_indexer = StringIndexer(inputCol="is_fraud", outputCol="label")

# Convert string columns to numeric using StringIndexer
indexers = [
    StringIndexer(inputCol=col, outputCol=col+"_index", handleInvalid="keep")
    for col in ['amount', 'customer_id_avrge_amount_1day', 'customer_id_avrge_amount_1week',
                'customer_id_avrge_amount_1month', 'customer_id_avrge_amount_3month',
                'customer_id_count_1day', 'customer_id_count_1week', 'customer_id_count_1month',
                'customer_id_count_3month', 'account_id_avrge_amount_1day', 'account_id_avrge_amount_1week',
                'account_id_avrge_amount_1month', 'account_id_avrge_amount_3month',
                'account_id_count_1day', 'account_id_count_1week', 'account_id_count_1month',
                'account_id_count_3month', 'transaction_in_weekend', 'transaction_at_night']
]

# Create a VectorAssembler to assemble the feature columns
feature_columns = [col+"_index" for col in ['amount', 'customer_id_avrge_amount_1day', 'customer_id_avrge_amount_1week',
                                            'customer_id_avrge_amount_1month', 'customer_id_avrge_amount_3month',
                                            'customer_id_count_1day', 'customer_id_count_1week',
                                            'customer_id_count_1month', 'customer_id_count_3month',
                                            'account_id_avrge_amount_1day', 'account_id_avrge_amount_1week',
                                            'account_id_avrge_amount_1month', 'account_id_avrge_amount_3month',
                                            'account_id_count_1day', 'account_id_count_1week', 'account_id_count_1month',
                                            'account_id_count_3month', 'transaction_in_weekend', 'transaction_at_night']
]
vectorAssembler = VectorAssembler(
    inputCols=feature_columns,
    outputCol="features"
)

# Split the data into training and testing sets
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

In [21]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import MultilayerPerceptronClassifier


# Step 2: Load the Keras model in Python
keras_model = load_model('./ml_model/mlp_fd_model.h5')

# Step 3: Convert the Keras model to a PySpark-compatible format
input_size = keras_model.input_shape[1]
output_size = keras_model.output_shape[1]

ml_model = MultilayerPerceptronClassifier(
    featuresCol='features',
    labelCol='label',
    layers=[input_size, 10, output_size]
)

assembler = VectorAssembler(inputCols=['features'], outputCol='features_vec')

pipeline = Pipeline(stages=[assembler, ml_model])
wrapped_model = pipeline.fit(spark.createDataFrame([(Vectors.dense([0.0]), 0.0)], ['features', 'label']))

ml_weights = [list(keras_model.layers[i].get_weights()[0].T.flatten()) for i in range(len(keras_model.layers))]
ml_biases = [list(keras_model.layers[i].get_weights()[1].flatten()) for i in range(len(keras_model.layers))]
ml_model.weights = ml_weights
ml_model.biases = ml_biases


Py4JJavaError: An error occurred while calling o111.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 4.0 failed 1 times, most recent failure: Lost task 5.0 in stage 4.0 (TID 12) (pyspark executor driver): java.lang.ArrayIndexOutOfBoundsException: arraycopy: last source index 19 out of bounds for double[1]
	at java.base/java.lang.System.arraycopy(Native Method)
	at org.apache.spark.ml.ann.DataStacker.$anonfun$stack$4(Layer.scala:665)
	at org.apache.spark.ml.ann.DataStacker.$anonfun$stack$4$adapted(Layer.scala:664)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.ml.ann.DataStacker.$anonfun$stack$3(Layer.scala:664)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.storage.memory.MemoryStore.putIterator(MemoryStore.scala:224)
	at org.apache.spark.storage.memory.MemoryStore.putIteratorAsValues(MemoryStore.scala:302)
	at 

In [4]:
# Step 4: Initialize a PySpark SparkSession
spark = SparkSession.builder.appName("MLP tf in PySpark").getOrCreate()

# Step 5: Prepare your input data as a PySpark DataFrame
input_data = spark.createDataFrame(...)

# Step 6: Transform the input data using the pre-trained model
transformed_data = wrapped_model.transform(input_data)

# Step 7: Retrieve the output from the transformed DataFrame
output_data = transformed_data.select("output_column_name")


In [30]:
data = spark.read.format("csv").option("header", "true").load("./ml_model/fraudulent_transactions_anonymous_ia.csv")

In [5]:
model_path = "./ml_model/mlp_fd_model.h5"
tf_transformer = TFModelTransformer.tfLoad(model_path)

Py4JJavaError: An error occurred while calling o25.load.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/home/jovyan/work/pyspark-elastic-ai/ml_model/mlp_fd_model.h5/metadata
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:292)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:288)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:292)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:288)
	at org.apache.spark.rdd.RDD.$anonfun$take$1(R

In [32]:
# Create a Multilayer Perceptron Classifier
layers = [len(feature_columns), 10, 5, 2]  # Adjust the number of neurons in each layer as needed
mlp = MultilayerPerceptronClassifier(
    labelCol="label",
    featuresCol="features",
    layers=layers,
    seed=42
)

In [ ]:
# Create a pipeline to combine the stages
pipeline = Pipeline(stages=[label_indexer] + indexers + [vectorAssembler, mlp])

# Split the data into training and testing sets
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

# Fit the pipeline on the training data
model = pipeline.fit(train_data)

In [ ]:
# Make predictions on the testing data
predictions = model.transform(test_data)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="accuracy"
)



In [ ]:
accuracy = evaluator.evaluate(predictions)
print("Accuracy: {:.2f}%".format(accuracy * 100))